# Transformer Application

## Configuration

### Hyperparameters

In [ ]:
from CustomLoss import CustomEmbeddingSliceLoss
import torch.nn as nn
import matplotlib.pyplot as plt

# HYPERPARAMETERS
BATCH_SIZE = 64
LEARNING_RATE = 0.0002

#transformer
NUM_HEADS = 90 # Dividers of 270: {1; 2; 3; 5; 6; 9; 10; 15; 18; 27; 30; 45; 54; 90; 135; 270}
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 5
DROPOUT=0.07

# Methods
loss_function_train = CustomEmbeddingSliceLoss(weight_deep_svg=10, weight_type=8.38, weight_parameters=2.11, weight_eos=0.937)
loss_function_val = CustomEmbeddingSliceLoss(weight_deep_svg=10, weight_type=0.91, weight_parameters=650, weight_eos=2.7)
# CONSTANTS
FEATURE_DIM = 270

## Load Prepared Tensors from Disk
Run file `prototype_dataset.ipynb` first

In [ ]:
import torch

train_sequence_input = torch.load('data/prototype_dataset/train_sequence_input.pt')
train_sequence_output = torch.load('data/prototype_dataset/train_sequence_output.pt')
test_sequence_input = torch.load('data/prototype_dataset/test_sequence_input.pt')
test_sequence_output = torch.load('data/prototype_dataset/test_sequence_output.pt')

In [ ]:
from prototype_dataset_helper import warn_if_contains_NaN

warn_if_contains_NaN(train_sequence_input)
warn_if_contains_NaN(train_sequence_output)
warn_if_contains_NaN(test_sequence_input)
warn_if_contains_NaN(test_sequence_output)

In [ ]:
print(train_sequence_input.size())
print(train_sequence_output.size())
print(test_sequence_input.size())
print(test_sequence_output.size())

## Build Dataloader with Batches

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

train_dataloader = DataLoader(TensorDataset(train_sequence_input.float(), train_sequence_output.float()),
                              batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_dataloader = DataLoader(TensorDataset(test_sequence_input.float(), test_sequence_output.float()),
                            batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
print(f"Train batches: {len(train_dataloader)}\n"
      f"Validation batches: {len(val_dataloader)}")

## Initialize

In [ ]:
# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

In [ ]:
from torch import nn
from AnimationTransformer import AnimationTransformer

model = AnimationTransformer(
    dim_model=FEATURE_DIM,
    num_heads=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dropout_p=DROPOUT,
    use_positional_encoder=False # No improvement
).to(device)

# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

total_param = sum(p.numel() for p in model.parameters())
print(f"The model has {total_param} trainable parameters")

## Training

In [ ]:
from AnimationTransformer import fit

train_loss_list, validation_loss_list, variance_list = fit(model,
                                            optimizer,
                                            loss_function_train,
                                            loss_function_val,
                                            train_dataloader,
                                            val_dataloader,
                                            epochs=10,
                                            device=device)

In [ ]:
# Define the number of additional epochs you want to train for
additional_epochs = 0

# Continue training the model for more epochs
new_train_loss, new_validation_loss, new_variance_list = fit(model,
                                          optimizer,
                                          loss_function_train,
                                          loss_function_val,
                                          train_dataloader,
                                          val_dataloader,
                                          epochs=additional_epochs,
                                          device=device)

# Extend the original loss lists with the new loss values
train_loss_list.extend(new_train_loss)
validation_loss_list.extend(new_validation_loss)
variance_list.extend(new_variance_list)

In [ ]:
def print_for_excel(list, title=""):
    print(f"{title};", ";".join([str(f"{element:.3f}").replace('.', ',') for element in list]))

In [ ]:
from CreativityLoss import dict_list_to_list_dict

print_for_excel(train_loss_list, title="Train Loss")
print_for_excel(validation_loss_list, title="Validation Loss")

print_dict = dict_list_to_list_dict(variance_list)
#print_for_excel(print_dict["batch_variance"], title="Batch Variance")
print_for_excel(print_dict["batch_variance_deep_svg"], title="Batch Variance (deep_svg)")
print_for_excel(print_dict["batch_variance_type"], title="Batch Variance (type)")
print_for_excel(print_dict["batch_variance_parameters"], title="Batch Variance (parameter)")
print_for_excel(print_dict["batch_variance_eos"], title="Batch Variance (eos)")
#print_for_excel(print_dict["sequence_variance"], title="Sequence Variance")
print_for_excel(print_dict["sequence_variance_deep_svg"], title="Sequence Variance (deep_svg)")
print_for_excel(print_dict["sequence_variance_type"], title="Sequence Variance (type)")
print_for_excel(print_dict["sequence_variance_parameters"], title="Sequence Variance (parameter)")
print_for_excel(print_dict["sequence_variance_eos"], title="Sequence Variance (eos)")

## Training and Validation Loss Plot

In [ ]:
# # Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(train_loss_list, label='Training Loss')
plt.plot(validation_loss_list, label='Validation Loss')

# Add title and labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Add a legend
plt.legend()

# Show the plot
plt.show()

# Prediction

In [ ]:
# Create a tensor of zeros with 270 elements
sos_token = torch.zeros(270)
# Set the value at the 256 index to 1
sos_token[269] = 1

In [ ]:
from AnimationTransformer import predict

predict(model, test_sequence_input[11], sos_token=sos_token, device=device, max_length=5, eos_threshold=0.8)

In [ ]:
def get_predicting_sequence_length(model, input_sequences, sos_token, device, max_length=5, eos_threshold=0.5):
    total_animations = 0
    for i in range(len(input_sequences)):
        sequence = predict(model, input_sequences[i], sos_token=sos_token, device=device, max_length=max_length, silent=True, eos_threshold=eos_threshold)
        total_animations += len(sequence) - 2
        print(f"{i}: {len(sequence)-2} Animations")
    
    print(f"In average {total_animations / len(input_sequences)} per sequence with eos_threshold {eos_threshold}")  
get_predicting_sequence_length(model, test_sequence_input[0:20], device=device, sos_token=sos_token, eos_threshold=0.8)

In [ ]:
# Sequences with long input
test_sequence_input[220, :, 0]

In [ ]:
# check sequence length / embedding
print(test_sequence_output[222, :, -14:])
print(test_sequence_input [222, :, -14:])

# Hyperparameter Tuning

In [ ]:
from AnimationTransformer import validation_loop, train_loop, creativity_loop
import optuna
from torch.utils.data import DataLoader

MAX_EPOCHS = 10

def objective(trial):
    # Define the hyperparameter search space
    learning_rate = trial.suggest_float('learning_rate', 0.000001, 0.002, log=True)
    num_encoder_layers = trial.suggest_categorical('num_encoder_layers', [3, 4, 5])
    num_decoder_layers = trial.suggest_categorical('num_decoder_layers', [5, 6, 7, 8])
    # batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    # num_heads = trial.suggest_categorical('num_heads', [9, 15, 27, 45, 90])
    dropout = trial.suggest_float('dropout', 0.06, 0.08)
    #use_positional_encoder = trial.suggest_categorical('pos_encoder_max_len', [True, False])
    
    loss_weight_type = trial.suggest_float('loss_weight_type', 4, 10, log=True)
    loss_weight_param = trial.suggest_float('loss_weight_param', 0.1, 3, log=True)
    loss_weight_eos = trial.suggest_float('loss_weight_eos', 0.01, 1, log=True)
    
    print(f'Parameters selected')
    print(f'loss_weight_type; loss_weight_param; loss_weight_eos')
    print(f'{loss_weight_type}; {loss_weight_param}; {loss_weight_eos}')
   
    # Instantiate the model with suggested hyperparameters
    model = AnimationTransformer(
        dim_model=FEATURE_DIM,
        num_heads=NUM_HEADS,
        num_encoder_layers=3,
        num_decoder_layers=5,
        dropout_p=0.07,
        use_positional_encoder=False
    ).to(device)
        
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
    
    loss_function_validation = CustomEmbeddingSliceLoss(weight_deep_svg=10,
                                                   weight_type=0.91,
                                                   weight_parameters=650,
                                                   weight_eos=2.7)
    loss_function_train = CustomEmbeddingSliceLoss(weight_deep_svg=10,
                                                   weight_type=loss_weight_type,
                                                   weight_parameters=loss_weight_param,
                                                   weight_eos=loss_weight_eos)
    
    train_loss_list = []
    validation_loss_list = []
    variance_list = []

    validation_loss = -1
    # Training loop with early stopping, validation, etc.
    for epoch in range(MAX_EPOCHS):
        print(f' =========== EPOCH {epoch} ===========')
        
        train_loss = train_loop(model, optimizer, loss_function_train, train_dataloader, device)
        train_loss_list += [train_loss]

        validation_loss = validation_loop(model, loss_function_validation, val_dataloader, device)
        validation_loss_list += [validation_loss]
        
        variance = creativity_loop(model, val_dataloader, device)
        variance_list.append(variance)
        
        print(f'Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}')
        
        # Report the validation loss to Optuna
        trial.report(validation_loss, epoch)
        
        # Implement early stopping logic
        if trial.should_prune():
            print(f"PRUNING IN EPOCH {epoch}")
            raise optuna.exceptions.TrialPruned()
    
    print(f'Best validation loss: {validation_loss}')
    print(f'loss_weight_type; loss_weight_param; loss_weight_eos')
    print(f'{loss_weight_type}; {loss_weight_param}; {loss_weight_eos}'.replace('.', ','))
    print_for_excel(train_loss_list, title="Train Loss")
    print_for_excel(validation_loss_list, title="Validation Loss")
    
    print_dict = dict_list_to_list_dict(variance_list)
    #print_for_excel(print_dict["batch_variance"], title="Batch Variance")
    print_for_excel(print_dict["batch_variance_deep_svg"], title="Batch Variance (deep_svg)")
    print_for_excel(print_dict["batch_variance_type"], title="Batch Variance (type)")
    print_for_excel(print_dict["batch_variance_parameters"], title="Batch Variance (parameter)")
    print_for_excel(print_dict["batch_variance_eos"], title="Batch Variance (eos)")
    #print_for_excel(print_dict["sequence_variance"], title="Sequence Variance")
    print_for_excel(print_dict["sequence_variance_deep_svg"], title="Sequence Variance (deep_svg)")
    print_for_excel(print_dict["sequence_variance_type"], title="Sequence Variance (type)")
    print_for_excel(print_dict["sequence_variance_parameters"], title="Sequence Variance (parameter)")
    print_for_excel(print_dict["sequence_variance_eos"], title="Sequence Variance (eos)")
    
    return validation_loss

Used:
- pick_and_animate_from_8     First Run
- pick_and_animate_from_8_v3  First Main Run
- pick_and_animate_from_8_loss_optimization
- pick_and_animate_from_8_loss_optimization_normalized
- 

In [ ]:
my_study = optuna.create_study(
    direction='minimize',
    study_name='pick_and_animate_from_8_v3', # IMPORTANT: Chance Name when new Dataset
    storage='sqlite:///animate_svg_optuna.db',
    load_if_exists=True
)

In [ ]:
#study = optuna.create_study(direction='minimize')
my_study.optimize(objective, n_trials=200)

In [ ]:
print("Best trial:")
trial = my_study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances

plot_optimization_history(my_study)

In [ ]:
plot_param_importances(my_study)

In [ ]:
from optuna.visualization import plot_slice

plot_slice(my_study)

In [ ]:
from optuna.visualization import plot_timeline

plot_timeline(my_study)